# High Level Coastal Model Test Bed (CMTB) SWASH 1D Example 

#### This notebook demonstrates CMTB integrated capabilities
    CMTB sets this up as a "skinny" 2D simulation that is effectively 1D
    Notebook users will have to download CMTB from github and install the required python packages
    Users select simulation settings in the User Input Cell and can walk through the rest of the code
    Once CMTB writes the input files, users can run SWASH separately in a command line interface 
    (commands listed at end of notebook) OR, if users have a SWASH mpiexec compiled (aka SWASH compiled in parallel mode),    
    users may run SWASH via CMTB

In [ ]:
import sys
sys.path.insert(0, '/Users/lszcz/Documents/CMTB/cmtb/')

from getdatatestbed import getDataFRF
from getdatatestbed.getDataFRF import getDataTestBed
from frontback import frontBackNEW
from prepdata import writeRunRead as wrr
from testbedutils import waveLib as sbwave
from plotting.operationalPlots import generate_CrossShoreTimeseries

import datetime as dt
import numpy as np
import os, glob, makenc
import matplotlib.pyplot as plt

## User Input Cell: Declare input settings

In [ ]:
workingDir = '/Users/lszcz/Documents/CMTB/cmtb/data'
exe = '/mirror/swash/swash.exe'
testName = 'swash1D_test_mpi'
versionPrefix='base'

# Select simulated period
startTime = dt.datetime(2019,10,11,15,0,0)
date_str = '2019-10-11'
endTime = dt.datetime(2019,10,11,16,0,0)
simulationDuration = 1
spinup = 900                                        # initial spinup time before model output (in seconds)
ncores = 18                                         # number of cores to use

# Select bathymetric setting
bathy_loc = 'survey'
profile_num = 945
xmin = 25
xmax = 915
dx = 1
dy = 1
fric_fac = 0.015

# Select sensor for boundary conditions
sensor = '8m-array'


## Auto-format additional variables 

In [ ]:
runDuration = (endTime - startTime).total_seconds()
ymin = profile_num - dy
ymax = profile_num + dy

## Create input dictionary 

In [ ]:
Inputs = {}
Inputs['THREDDS'] = 'CHL'
Inputs['bathyLoc'] = 'integrated_bathy'
Inputs['startTime'] = dt.datetime.strftime(startTime,'%Y-%m-%dT%H:%M:%SZ')
Inputs['endTime'] = dt.datetime.strftime(endTime,'%Y-%m-%dT%H:%M:%SZ')
Inputs['modelSettings'] = {}
Inputs['modelSettings']['version_prefix'] = 'base'
Inputs['modelSettings']['yBounds'] = [ymin, ymax+1]
Inputs['modelSettings']['grid'] = '1D'
Inputs['modelSettings']['modelName'] = 'swash'
Inputs['profileNumber'] = profile_num
Inputs['simulationDuration'] = simulationDuration
Inputs['modelExecutable'] = '/Users/lszcz/Documents/CMTB/cmtb/data/swash.exe'
Inputs['workingDirectory'] = '/data'
Inputs['netCDFdir'] = '/data'
Inputs['logfileLoc'] = 'data'
Inputs['plotFlag'] = False
Inputs['generateFlag'] = True
Inputs['runFlag'] = True
Inputs['analyzeFlag'] = False

## Gather data

In [ ]:
go = getDataFRF.getObs(startTime,endTime)     # initialize go command
wave_spec = go.getWaveData(sensor,spec=True)  # grab raw spectra
wl = go.getWL()                               # grab raw water levels
gdTB = getDataTestBed(startTime, endTime)     # initialize gdTB command
rawwind = go.getWind(gaugenumber=0)

## Initialize swio

In [ ]:
swio = wrr.swashIO(workingDirectory=workingDir, testName=testName,
                   versionPrefix=versionPrefix, startTime=startTime,
                   simulatedRunTime=simulationDuration,
                   endTime=endTime, runFlag=Inputs['runFlag'],
                   generateFlag=Inputs['generateFlag'], readFlag=Inputs['analyzeFlag'],
                   newModelParams=Inputs['modelSettings'])

## Get AND prep bathy, prep wave and waterlevel packets formatted for SWASH input

In [ ]:
wavePacket, windPacket, wlPacket, bathyPacket, swio = frontBackNEW.swashSimSetup(Inputs['startTime'],
                                                        inputDict=Inputs,
                                                        allWL=wl,
                                                        allWind=rawwind,
                                                        allWave=wave_spec,
                                                        wrr=swio)

## Write SWASH files

In [ ]:
swio.writeAllFiles(wavePacket=wavePacket,wlPacket=wlPacket,
                                  bathyPacket=bathyPacket)

## -----------------------------------------------------------------------------------------
## Run SWASH

In [ ]:
# overwriting below to account for windows running ubuntu shell on windows machine
swio.runString1 = 'wsl mpiexec -n 18 /mirror/swash/swash.exe INPUT' 

In [ ]:
swio.runSimulation(exe)

## -----------------------------------------------------------------------------------------
## Load in results

In [ ]:
os.chdir(swio.workingDirectory)                               # add path to results .mat file to directory
[dataDict, metaDict] = swio.loadSwash_Mat(testName+'.mat')    # load in formatted results
    # produces a benign warning

In [ ]:
fspec, freqs = sbwave.timeSeriesAnalysis1D(dataDict['time'].squeeze(), dataDict['eta'].squeeze(), bandAvg=6)
Stats = sbwave.stats1D(fspec=fspec, frqbins=freqs, lowFreq=None, highFreq=None)
HsTS = 4 * np.std(dataDict['eta'].squeeze(), axis=0)

In [ ]:
plt.figure()
plt.plot(dataDict['xFRF'],HsTS)
plt.ylabel('$H_s$ (m)')
plt.xlabel('x (m)')
plt.show()

In [ ]:
generate_CrossShoreTimeseries(workingDir+'/'+testName+'/img', np.mean(dataDict['eta'].squeeze(), axis=0), dataDict['elevation'], dataDict['xFRF'])